In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [14]:
url = "https://en.wikipedia.org/wiki/List_of_female_Academy_Award_winners_and_nominees_for_non-gendered_categories"
page = requests.get( url )
soup = BeautifulSoup(page.content, 'html.parser')

#### Fazer um textinho bonitinho explicando o que rolou

In [303]:
year = []
category = []
status = []
all_tables = soup.find_all(class_='wikitable')

for table in all_tables:
	td = table.find_all('td')
	new_year = 0
	for i in range(0, len(td)):
		try:
			new_year = int(td[i].text[0:4])
		except ValueError:
			pass
		if td[i].get('class'):
			status.append(td[i].text[0:-1])
			category.append(td[0].text[18:-1])
			if new_year != 0:
				year.append(new_year)

oscar_women_df = pd.DataFrame({
	'year': year,
	'category': category,
	'status': status
})

oscar_women_df

,year,category,status
0,2007,Best Animated Feature,Nominated
1,2011,Best Animated Feature,Nominated
2,2012,Best Animated Feature,Won
3,2013,Best Animated Feature,Won
4,2013,Best Animated Feature,Nominated
...,...,...,...
1418,2017,Best Original Screenplay,Nominated
1419,2017,Best Original Screenplay,Nominated
1420,2018,Best Original Screenplay,Nominated
1421,1917,Best Original Screenplay,Nominated


In [ ]:
table = all_tables[0]
td = table.find_all('td')
td[14].text

'Won\n'

In [ ]:
categoria = soup.find(id="Best_Animated_Feature")
categoria

<span class="mw-headline" id="Best_Animated_Feature">Best Animated Feature</span>